In [1]:
import pandas as pd
import tensorflow as tf
from PIL import Image
import numpy as np
import os

def img_to_address(screenshot):
    width, height = screenshot.size
    glyph_side = width / 60
    left = glyph_side * (1/3)
    top = height-(2*glyph_side)
    right = glyph_side*(12 + (1/3))
    bottom = height-glyph_side
    return screenshot.crop((left, top, right, bottom))

def color_to_binary(im_addr):
    """converts color address to black/white 

    dynamically attempts to adjust threshold where the first attempt is too dark or too light
    """
    threshold = 200    
    # threshold_increment = .05
    min_ratio = 15
    max_ratio = 20
    min_threshold = 128
    max_threshold = 254
    # Convert color space
    # Grayscale (probably unnecessary)
    im_addr_gray = im_addr.convert("L")   
    # Binary (definitely excessive)
    im_addr_binary= im_addr_gray.point( lambda p: 255 if p > threshold else 0 )
    hist = im_addr_binary.histogram()
    # Simple, clumsy, dynamic threshold adjustment
    try:
        ratio = hist[0]/(hist[-1]+1)
        while ratio > max_ratio or ratio < min_ratio:
            if (ratio > max_ratio): # Too Dark
                # threshold *= (1-threshold_increment)
                threshold -= 1
                # print("Too Dark", end='\t')
                # print('Threshold decreased to: {} '.format(str(ceil(threshold))), end='\t')
            else:                
                # threshold *= (1+threshold_increment)
                threshold += 1
                # print("Too Light", end='\t')
                # print('Threshold increased to: {} '.format(str(ceil(threshold))), end='\t')
            if threshold < min_threshold:
                # print("Threshold too low")
                return im_addr_gray
            if threshold > max_threshold:
                # print("Threshold too high")
                return im_addr_gray
            im_addr_binary= im_addr_gray.point( lambda p: 255 if p > threshold else 0 )
            hist = im_addr_binary.histogram()
            ratio = hist[0]/(hist[-1]+1)
            # print(f'New ratio: {ratio}')
        return im_addr_gray.point( lambda p: 255 if p > 200 else 0 )
    except ZeroDivisionError:
        print("B/(W+1) Ratio raised zero division error unexpectedly.")
        return im_addr_gray


def isolate_glyph(img, ind):
	'''Isolates one of 12 glyphs in a 768x64 addr image
	'''
	# Setting the points for cropped image
	left = 0 + ind*64
	right = left+64
	top = 0
	bottom = 64
	
	# Cropped image of above dimension
	# (It will not change original image)
	return img.crop((left, top, right, bottom))



def extract_glyphs(screenshot):
    """Fetches an screenshot url and extracts glpyhs, returning a 12-character hex string
    """

    # Default parameters
    # null_address = "000000000000"       # location of central black hole 
    # target_aspect_ratio = 16/9
    addr_size = (12*64, 64) 

    
    # width, height = screenshot.size
    # if abs((width / height) - target_aspect_ratio)>.01:
    #     note=f'Improper aspect ratio. {width}x{height}'
    #     return null_address       


    # Crop to just address portion (to avoid resizing everything to 4k)
    im_addr = img_to_address(screenshot)
    im_addr.save('temp_img\\address.png')
    
    # Preprocess Image
    
    # Resize address to be the size it is with 4k captures
    im_addr = im_addr.resize(size=addr_size, resample=Image.Resampling.LANCZOS)
    im_addr.save('temp_img\\address_resize.png')

    # Convert color space to binary (ish)
    im_addr = color_to_binary(im_addr)
    im_addr.save('temp_img\\address_binary.png')
    # return(im_addr)
    
    # Start empty address string
    glyph_array = []
    path_array = []

    # Loop through 12 glyphs
    # ! This was a terrible way to do this 
    # In the future, just store batches of images and classify a batch at a time
    for glyph_num in range(12):
        
        # Crop glyph
        glyph = isolate_glyph(im_addr, glyph_num)    
        
        temp_path = f"temp_img\\glyph_{glyph_num}.png"
        glyph.save(temp_path)
        
        glyph_array.append(glyph)  
        path_array.append(temp_path)
        
        # Save glyph temporarily

    return(glyph_array, path_array)
        

def classify_glyph_array(paths, classifier):
    null_address = "000000000000"       # location of central black hole 
    img_height = img_width = 64
    class_names = ["0","1","2","3","4","5","6","7","8","9","A","B","C","D","E","F","u"]
    address = ""
    glyphs = []

    for path in paths:
        tf_glyph =  tf.keras.utils.load_img(path, target_size=(img_height, img_width))
        img_array = tf.keras.utils.img_to_array(tf_glyph)
        img_array = tf.expand_dims(img_array, axis=0) # Create a batch
        glyphs.append(img_array)


    for glyph in glyphs:
        # Classify Glyph
        predictions = classifier.predict(glyph, verbose=0,)
        score = tf.nn.softmax(predictions[0])
        prediction = class_names[np.argmax(score)]
        address += prediction
    
    if 'u' in address:
        return null_address

    return address

In [2]:
classifier = tf.keras.models.load_model('C:\\Users\\derek\\Dropbox\\Back to School\\Chico State\\Courses\\2022 Fall\\CSCI 490 Capstone\\capstone-nms-galactic-map\\production_scripts\\models\\glyphs_17')

my_screenshots_folder = "C:\\csci490\\images\\my_screenshots"
my_screenshot_paths = os.listdir(my_screenshots_folder)

my_addresses = []

for i in range(0, len(my_screenshot_paths)):
# for i in range(0, 1):
    # check if the fname ends with png
    fname = my_screenshot_paths[i]
    path = my_screenshots_folder + "\\" + fname
    if (fname.endswith(".jpg")):
        screenshot = Image.open(path)
        glyph_array, paths = extract_glyphs(screenshot)
        # print(paths)
        my_address = classify_glyph_array(paths, classifier)
        my_addresses.append(my_address)

print(my_addresses)



['10BD10102222', '10BD10102222', '10F810102222', '614A10103222', '614A10103222', '614A10103222', '100B0F101227', '200B0F101227', '200B0F101227', '51770F101227', '51770F101227', '115012105229', '10C11710522C', '40C11710522C']


In [3]:
# Suppress 'SettingWithCopyWarning: A value is trying to be set on a copy of a slice from a DataFrame.'
pd.options.mode.chained_assignment = None  # default='warn'

from math import sqrt, floor

df = pd.DataFrame(my_addresses, columns =['address'])

df['body_index_h']     = df['address'].str[0]
df['sys_index_h']      = df['address'].str[1:4]
df['region_y_h']       = df['address'].str[4:6]
df['region_x_h']       = df['address'].str[6:9]
df['region_z_h']       = df['address'].str[9:12]

df['planet_index'] = df.body_index_h.apply(lambda x: int(x, 16))
df['sys_index']    = df.sys_index_h.apply(lambda x: int(x, 16))
df['y']            = df.region_y_h.apply(lambda x: int(x, 16) if int(x, 16) < 128 else int(x, 16)-255)
df['x']            = df.region_x_h.apply(lambda x: int(x, 16) if int(x, 16) < 2048 else int(x, 16)-4095)
df['z']            = df.region_z_h.apply(lambda x: int(x, 16) if int(x, 16) < 2048 else int(x, 16)-4095)
df['k_ly_from_center'] = df.apply(lambda x: floor(sqrt(x.x*x.x + x.y*x.y + x.z*x.z)*.4), axis=1)

df

,address,body_index_h,sys_index_h,region_y_h,region_x_h,region_z_h,planet_index,sys_index,y,x,z,k_ly_from_center
0,10BD10102222,1,0BD,10,102,222,1,189,16,258,546,241
1,10BD10102222,1,0BD,10,102,222,1,189,16,258,546,241
2,10F810102222,1,0F8,10,102,222,1,248,16,258,546,241
3,614A10103222,6,14A,10,103,222,6,330,16,259,546,241
4,614A10103222,6,14A,10,103,222,6,330,16,259,546,241
5,614A10103222,6,14A,10,103,222,6,330,16,259,546,241
6,100B0F101227,1,00B,0F,101,227,1,11,15,257,551,243
7,200B0F101227,2,00B,0F,101,227,2,11,15,257,551,243
8,200B0F101227,2,00B,0F,101,227,2,11,15,257,551,243
9,51770F101227,5,177,0F,101,227,5,375,15,257,551,243


In [11]:
df[['address','planet_index','sys_index','x','y','z','k_ly_from_center']]

,address,planet_index,sys_index,x,y,z,k_ly_from_center
0,10BD10102222,1,189,258,16,546,241
1,10BD10102222,1,189,258,16,546,241
2,10F810102222,1,248,258,16,546,241
3,614A10103222,6,330,259,16,546,241
4,614A10103222,6,330,259,16,546,241
5,614A10103222,6,330,259,16,546,241
6,100B0F101227,1,11,257,15,551,243
7,200B0F101227,2,11,257,15,551,243
8,200B0F101227,2,11,257,15,551,243
9,51770F101227,5,375,257,15,551,243


In [4]:
df.to_csv('my_star_systems.csv')

In [5]:
classifier.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 rescaling_3 (Rescaling)     (None, 64, 64, 3)         0         
                                                                 
 conv2d_6 (Conv2D)           (None, 62, 62, 32)        896       
                                                                 
 max_pooling2d_6 (MaxPooling  (None, 31, 31, 32)       0         
 2D)                                                             
                                                                 
 conv2d_7 (Conv2D)           (None, 29, 29, 32)        9248      
                                                                 
 max_pooling2d_7 (MaxPooling  (None, 14, 14, 32)       0         
 2D)                                                             
                                                                 
 conv2d_8 (Conv2D)           (None, 12, 12, 32)       

In [8]:
from keras.utils.vis_utils import plot_model
import graphviz
import pydot

In [10]:
# plot_model(classifier, show_shapes=True, show_layer_names=True)
plot_model(classifier, to_file='model_plot.png', show_shapes=True, show_layer_names=True)


You must install pydot (`pip install pydot`) and install graphviz (see instructions at https://graphviz.gitlab.io/download/) for plot_model to work.
